In [1]:
from datasets import load_dataset
import pandas as pd
ds = load_dataset("Sri1999/creditmix-dataset")
data_dup = pd.DataFrame(ds["train"])
data_dup.answer.value_counts()

d:\anaconda\envs\unsloth_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


answer
Standard    14591
Good         9689
Bad          7588
Name: count, dtype: int64

In [2]:
from datasets import Dataset
import random

def rebalance(ds):
    groups = {"Good": [], "Bad": [], "Standard": []}
    for row in ds:
        groups[row["answer"]].append(row)
    min_len = min(len(groups["Good"]), len(groups["Bad"]))
    balanced = groups["Good"][:min_len] + groups["Bad"][:min_len] + random.sample(groups["Standard"], min_len)
    random.shuffle(balanced)
    return Dataset.from_list(balanced)

ds_balanced = rebalance(ds["train"])
pd.DataFrame(ds_balanced).answer.value_counts()

answer
Good        7588
Standard    7588
Bad         7588
Name: count, dtype: int64

In [3]:
SYSTEM_PROMPT = """
You are a financial risk analysis assistant.

Respond in the following format:
<reasoning>
(your reasoning here)
</reasoning>
<answer>
Choose exactly one of: "Good", "Bad", or "Standard"
</answer>
"""


# Convert to GSM8K-like format
ds = ds_balanced.map(lambda x: {
    "question": x["question"],
    "answer": x["answer"],
    "prompt": [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": x["question"]}
    ]
})
ds[0]

Map: 100%|██████████| 22764/22764 [00:00<00:00, 24998.90 examples/s]


{'question': 'Age: 46.0, Occupation: Musician, Annual Income: 80633.22, Outstanding Debt: 295.42, Credit Utilization Ratio: 24.881642105939, Payment Behaviour: High_spent_Large_value_payments',
 'answer': 'Good',
 'prompt': [{'content': '\nYou are a financial risk analysis assistant.\n\nRespond in the following format:\n<reasoning>\n(your reasoning here)\n</reasoning>\n<answer>\nChoose exactly one of: "Good", "Bad", or "Standard"\n</answer>\n',
   'role': 'system'},
  {'content': 'Age: 46.0, Occupation: Musician, Annual Income: 80633.22, Outstanding Debt: 295.42, Credit Utilization Ratio: 24.881642105939, Payment Behaviour: High_spent_Large_value_payments',
   'role': 'user'}]}

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
lora_rank = 32

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-3B-Instruct",
    load_in_4bit=False,
    max_seq_length=max_seq_length,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W1012 14:53:58.807000 22464 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


d:\anaconda\envs\unsloth_env\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:341: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.9.9: Fast Qwen2 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.54s/it]


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r                     = lora_rank,
    lora_alpha            = lora_rank,
    lora_dropout          = 0.05,
    target_modules        = ["q_proj", "v_proj"],   # ← per Reference 1 fix
    use_gradient_checkpointing = "unsloth",
    random_state          = 3407,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.9.9 patched 36 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [6]:
import re

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    # Extract model responses (what the LLM generated)
    responses = [completion[0]['content'] for completion in completions]

    # Get the question (last user message in prompt)
    q = prompts[0][-1]['content']

    # Extract only <answer>...</answer> from each response
    extracted_responses = [extract_xml_answer(r) for r in responses]

    # Debug print to see what's happening
    print(
        '-'*20,
        f"\nQuestion:\n{q}",
        f"\nGround Truth:\n{answer[0]}",
        f"\nModel Raw Response:\n{responses[0]}",
        f"\nExtracted Answer:\n{extracted_responses[0]}"
    )

    # Reward = 2.0 if correct label, else 0.0
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]


VALID_LABELS = {"Good", "Bad", "Standard"}

def cat_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]

    # Reward 0.5 if the model produces *any valid label*, else 0.0
    return [0.5 if r in VALID_LABELS else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]
def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]


In [7]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",

    logging_steps=1,
    per_device_train_batch_size=6,   # <-- multiple of num_generations
    gradient_accumulation_steps=1,
    num_generations=6,               # <-- pairs with batch size above

    max_prompt_length=max_prompt_length,
    max_completion_length=max_seq_length - max_prompt_length,

    max_steps=100,
    save_steps=25,                   # frequent saves
    save_total_limit=2,              # keep space in outputs/
    max_grad_norm=0.1,
    report_to="none",
    output_dir="outputs",
)

In [8]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        cat_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = ds,
)

In [9]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 22,764 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 7,372,800 of 3,093,311,488 (0.24% trained)


-------------------- 
Question:
Age: 32.0, Occupation: Writer, Annual Income: 29895.53, Outstanding Debt: 878.78, Credit Utilization Ratio: 30.34030690810204, Payment Behaviour: High_spent_Large_value_payments 
Ground Truth:
Good 
Model Raw Response:
<reasoning>
Based on the information provided, the individual has a relatively low outstanding debt and a moderate credit utilization ratio. However, the age of 32 might not initially suggest significant risk, but it's important to consider the age in the context of the other factors. Occupation as a writer generally suggests a stable, if not necessarily high-income, profession. The annual income is satisfactory, but not overwhelming. The credit utilization ratio is moderate, which is generally considered healthy, but comes with the caveat that it is specifically high with respect to large value payments, which might lead to more scrutiny by creditors.

Payment behavior ("High_spent_Large_value_payments") suggests a potentially high-risk p

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / xmlcount_reward_func / mean,rewards / xmlcount_reward_func / std,rewards / soft_format_reward_func / mean,rewards / soft_format_reward_func / std,rewards / strict_format_reward_func / mean,rewards / strict_format_reward_func / std,rewards / cat_reward_func / mean,rewards / cat_reward_func / std,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std
1,0.000000,0.185333,0.650351,232.833344,140.000000,331.000000,0.000000,232.833344,140.000000,331.000000,0.000000,-0.648000,0.361510,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.333333,0.816497
2,-0.000000,-0.052500,0.156777,217.166672,187.000000,257.000000,0.000000,217.166672,187.000000,257.000000,0.000000,-0.552500,0.156777,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000
3,0.000000,0.000667,0.292856,226.500000,165.000000,319.000000,0.000000,226.500000,165.000000,319.000000,0.001516,-0.499333,0.292856,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000
4,0.000000,0.505167,0.974612,252.500000,155.000000,315.000000,0.000000,252.500000,155.000000,315.000000,0.004329,-0.661500,0.277132,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.666667,1.032796
5,0.000000,0.794833,1.114589,245.166672,180.000000,300.000000,0.000000,245.166672,180.000000,300.000000,0.007986,-0.705167,0.206662,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,1.000000,1.095445
6,0.000000,-0.327833,0.424197,279.666687,163.000000,424.000000,0.000000,279.666687,163.000000,424.000000,0.003257,-0.827833,0.424197,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000
7,0.000000,0.767333,0.968421,187.000000,123.000000,294.000000,0.000000,187.000000,123.000000,294.000000,0.002028,-0.399333,0.340940,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.666667,1.032796
8,0.000000,1.360667,1.129219,188.666672,150.000000,245.000000,0.000000,188.666672,150.000000,245.000000,0.005384,-0.139333,0.317389,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,1.000000,1.095445
9,0.000000,0.806833,1.208925,245.000000,169.000000,324.000000,0.000000,245.000000,169.000000,324.000000,0.004822,-0.693167,0.262868,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,1.000000,1.095445
10,0.000000,1.567000,0.867637,181.666672,119.000000,256.000000,0.000000,181.666672,119.000000,256.000000,0.004612,-0.266333,0.322894,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,1.333333,1.032796


Unsloth: Will smartly offload gradients to save VRAM!
-------------------- 
Question:
Age: 53.0, Occupation: Journalist, Annual Income: 170824.68, Outstanding Debt: 1167.04, Credit Utilization Ratio: 38.21497751057565, Payment Behaviour: High_spent_Small_value_payments 
Ground Truth:
Good 
Model Raw Response:
<reasoning>
Given the data, the credit utilization ratio of 38.21% is within a relatively good range, typically recommended to be under 20%. The journalist's annual income of $170,824.68 is quite high, which suggests they have the means to manage credit responsibly and may even have some savings. Their credit utilization ratio, while not spectacularly low, is still in a solid range and doesn't appear to pose a significant risk.

Their payment behaviour (described as "High_spent_Small_value_payments") does raise some concern. While the actual meanings of these terms are not specified, larger payments and their frequency can be a sign that the individual has flexibility in managing 

TrainOutput(global_step=100, training_loss=6.997105219022082e-06, metrics={'train_runtime': 5561.537, 'train_samples_per_second': 0.108, 'train_steps_per_second': 0.018, 'total_flos': 0.0, 'train_loss': 6.997105219022082e-06})

In [13]:
import os 
model.push_to_hub(
    "Sri1999/Qwen2.5-3B-Instruct-CreditMix-LORA",  
    tokenizer, 
    token = os.environ["HF_ACCESS_TOKEN"]
)

adapter_model.safetensors: 100%|██████████| 29.5M/29.5M [00:17<00:00, 1.69MB/s]


Saved model to https://huggingface.co/Sri1999/Qwen2.5-3B-Instruct-CreditMix-LORA


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch, os

# === Paths ===
base_id   = "Qwen/Qwen2.5-3B-Instruct"                 
lora_ckpt = r"D:\Narwal\fine-tuning-lora-qlora\outputs\checkpoint-100"  # your QLoRA checkpoint
out_dir   = r"D:\Narwal\fine-tuning-lora-qlora\merged-qwen2.5-3B-lora"       # merged output

os.makedirs(out_dir, exist_ok=True)

# === Load tokenizer ===
tok = AutoTokenizer.from_pretrained(base_id, trust_remote_code=True)

# === Load base model in FP16 ===
model = AutoModelForCausalLM.from_pretrained(
    base_id,
    dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

# === Attach your QLoRA adapter ===
model = PeftModel.from_pretrained(model, lora_ckpt)

# === Merge the LoRA weights into the base model ===
model = model.merge_and_unload()

# === Save the merged model ===
model.save_pretrained(out_dir, safe_serialization=True)
tok.save_pretrained(out_dir)

print(f"✅ Merged model saved to: {out_dir}")

d:\anaconda\envs\unsloth_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1012 17:25:16.843000 58936 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.24s/it]


✅ Merged model saved to: D:\Narwal\fine-tuning-lora-qlora\merged-qwen2.5-3B-lora


In [7]:
from llama_cpp import Llama

# 1️⃣ Load the model once
llm = Llama(
    model_path=r"D:\Narwal\fine-tuning-lora-qlora\qwen2.5-3b--lora-f16.gguf",
    n_ctx=2048,            # match your fine-tuning context
    n_threads=8,           # adjust to CPU cores
    n_batch=512,
    verbose=False
)

# 2️⃣ Define the persona-style inference function
def ask_financial_risk(question: str,
                       max_tokens: int = 256,
                       generation_config: dict | None = None) -> str:
    """
    Streams a structured financial-risk analysis reply from your fine-tuned model.
    """
    if generation_config is None:
        generation_config = {
            "temperature": 0.7,
            "top_p": 0.9,
            "min_p": 0.0,
            "frequency_penalty": 0.0,
            "presence_penalty": 0.0,
            "repeat_penalty": 1.1,
            "top_k": 40
        }

    full_text = ""
    prompt = f"""You are a financial risk analysis assistant.
    Respond in the following format:
    <reasoning>
    (your reasoning here)
    </reasoning>
    <answer>
    Choose exactly one of: "Good", "Bad", or "Standard"
    </answer>

{question}
"""
    for chunk in llm.create_completion(
        prompt,
        max_tokens=max_tokens,
        stream=True,
        temperature=generation_config["temperature"],
        top_p=generation_config["top_p"],
        min_p=generation_config["min_p"],
        frequency_penalty=generation_config["frequency_penalty"],
        presence_penalty=generation_config["presence_penalty"],
        repeat_penalty=generation_config["repeat_penalty"],
        top_k=generation_config["top_k"],
    ):
        text_piece = chunk["choices"][0]["text"]
        print(text_piece, end="", flush=True)   # live stream to console
        full_text += text_piece

    print()
    return full_text

# 3️⃣ Example test query
if __name__ == "__main__":
    ask_financial_risk(
        "Age: 22.0, Occupation: Developer, Annual Income: 35547.71, Outstanding Debt: 943.86, Credit Utilization Ratio: 31.65235104376109, Payment Behaviour: High_spent_Medium_value_payments"
    )
""

llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


<reasoning>
The given individual is a 22-year-old developer with an annual income of $35,547.71 and currently has outstanding debt amounting to $943.86. Their credit utilization ratio stands at 31.65%, which is generally considered good as it indicates they are using only a small portion of their available credit limit. The payment behavior described as "High_spent_Medium_value_payments" might be concerning, suggesting that they frequently spend money but the value of transactions isn't exceptionally high or low. It's important to balance spending and saving habits for long-term financial health.
</reasoning>
(answer: Standard) 
The given individual presents a typical profile with an average age, moderate income, manageable credit utilization ratio, and varied payment behavior that doesn't indicate either extremely risky or excessively conservative money management patterns. Therefore, the risk assessment falls within the 'Standard' range. </answer>


''